In [1]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

from pathlib import Path
from datetime import datetime

# TODO: Create PyNilm python package
import sys
sys.path.append('../') # enabling pynilm import (src content)
sys.path.append('../extras/nilmtk-contrib')

In [2]:
# That's our greatest contribution!!!
from pynilm.data import DataWrapper
from pynilm.experiment import Experiment
from pynilm.models import ConvNet
from pynilm.preprocessing import (
    RQATransformer,
    SequenceToImageTransformer, 
    DeepLearningTransformer
)
from pynilm.settings import (
    SAMPLE_RATE,
    APPLIANCES,
    RP_PARAMS,
    INPUT_SHAPE,
    ESTIMATORS,
    EVALUATION_METRICS
)

In [ ]:
# Models
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
from pyts.image import (
    RecurrencePlot, 
    GramianAngularField, 
    # MarkovTransitionField
)

In [ ]:
# https://dl.acm.org/doi/10.1145/3360322.3360844
# Seq2Point - https://arxiv.org/pdf/1612.09106.pdf
# https://link.springer.com/epdf/10.1007/s00521-020-04916-5?sharing_token=bTZg6CBADDbWx7UVvztexPe4RwlQNchNByi7wbcMAY4YyOCPZ8jI-u3LyC4lDtEOZIQACACm_MVY_633J4jzg0CtjGEkhvPkzOs5Z-2UGgB1P_m1_4nDnPxtIplmNRaDx7TM52V6MVQYVJPSqJEKpxv1n3RqXoEm1ZpW5amjaaA=

for window_size in [
    int((1 * 60) / SAMPLE_RATE), # 1 min
    int((5 * 60) / SAMPLE_RATE), # 5 mins
    int((10 * 60) / SAMPLE_RATE), # 10 mins
    int((15 * 60) / SAMPLE_RATE), # 15 mins
    int((30 * 60) / SAMPLE_RATE), # 30 mins
    int((45 * 60) / SAMPLE_RATE), # 45 mins
    int((1 * 60 * 60) / SAMPLE_RATE), # 1hour
    
    # Empirical tests
    90,1080,60,2040,
    720, # 900
]:
    exp = Experiment(
        model_type='binary',
        models={
            
            'RQA-MLP': Pipeline([
                (
                    'feature_extractor', RQATransformer(
                        params=RP_PARAMS,
                        model_columns=[
                            "Recurrence rate (RR)", 
                            "Determinism (DET)"
                        ]
                    )
                ),
                ('scaler', StandardScaler()),
                ('estimator', clone(ESTIMATORS['MLP']))
            ]),
            
            'RP-CNN': Pipeline([
                ('rp', SequenceToImageTransformer(
                    image_transformer=RecurrencePlot(**RP_PARAMS),
                    image_shape=INPUT_SHAPE,
                    scaling=True,
                    flatten_output=False
                    )
                ),
                (
                    'estimator', ConvNet(
                        input_shape=INPUT_SHAPE,
                        output_dim=1,
                        loss_function='binary_crossentropy',
                        metrics=['accuracy'],
                        output_activation='sigmoid',
                        fit_params={
                            'epochs': 10,
                            'batch_size': 64,
                            'verbose': 1
                        }
                    )
                )
            ]),
            
            'GASF-CNN': Pipeline([
                ('rp', SequenceToImageTransformer(
                    image_transformer=GramianAngularField(
                        method='summation',
                        ),
                    image_shape=INPUT_SHAPE,
                    scaling=True,
                    flatten_output=False
                    )
                ),
                (
                    'estimator', ConvNet(
                        input_shape=INPUT_SHAPE,
                        output_dim=1,
                        loss_function='binary_crossentropy',
                        metrics=['accuracy'],
                        output_activation='sigmoid',
                        fit_params={
                            'epochs': 10,
                            'batch_size': 64,
                            'verbose': 1
                        }
                    )
                )
            ]),
            
            'GADF-CNN': Pipeline([
                ('rp', SequenceToImageTransformer(
                    image_transformer=GramianAngularField(
                        method='difference'
                        ),
                    image_shape=INPUT_SHAPE,
                    scaling=True,
                    flatten_output=False
                    )
                ),
                (
                    'estimator', ConvNet(
                        input_shape=INPUT_SHAPE,
                        output_dim=1,
                        loss_function='binary_crossentropy',
                        metrics=['accuracy'],
                        output_activation='sigmoid',
                        fit_params={
                            'epochs': 10,
                            'batch_size': 64,
                            'verbose': 1
                        }
                    )
                )
            ]),
            
            'RP-VGG16-MLP': Pipeline([
                ('rp', SequenceToImageTransformer(
                    image_transformer=RecurrencePlot(**RP_PARAMS),
                    image_shape=INPUT_SHAPE,
                    scaling=True,
                    flatten_output=False
                    )
                ),
                ('dl', DeepLearningTransformer(
                    model='VGG16',
                    weights='imagenet',
                    pooling='avg'
                    )
                ),
                ('estimator', clone(ESTIMATORS['MLP']))
            ]),
            
            'GASF-VGG16-MLP': Pipeline([
                ('rp', SequenceToImageTransformer(
                    image_transformer=GramianAngularField(
                        method='summation',
                        ),
                    image_shape=INPUT_SHAPE,
                    scaling=True,
                    flatten_output=False
                    )
                    ),
                ('dl', DeepLearningTransformer(
                    model='VGG16',
                    weights='imagenet',
                    pooling='avg'
                    )
                ),
                ('estimator', clone(ESTIMATORS['MLP']))
            ]),
            
            'GADF-VGG16-MLP': Pipeline([
                ('rp', SequenceToImageTransformer(
                    image_transformer=GramianAngularField(
                        method='difference'
                        ),
                    image_shape=INPUT_SHAPE,
                    scaling=True,
                    flatten_output=False
                    )
                    ),
                ('dl', DeepLearningTransformer(
                    model='VGG16',
                    weights='imagenet',
                    pooling='avg'
                    )
                ),
                ('estimator', clone(ESTIMATORS['MLP']))
            ]),
            
            },
        
        train_datasource=[
            DataWrapper(
                dataset_path='../data/redd.h5',
                building=3, 
                appliances=APPLIANCES,
                sample_period=SAMPLE_RATE,
                start='2011-04-16 00:00:00',
                end='2011-05-16 23:59:59',
                return_mode='dataframe',
                windows_size=window_size,
                windows_stride=window_size, # No overlapping!,
                get_activations=True,
                activations_type=int,
                debug=False
            )
        ],
        
        test_datasource=[
            DataWrapper(
                dataset_path='../data/redd.h5',
                building=3, 
                appliances=APPLIANCES,
                sample_period=SAMPLE_RATE,
                start='2011-05-17 00:00:00',
                end='2011-05-31 23:59:59',
                return_mode='dataframe',
                windows_size=window_size,
                windows_stride=window_size, # No overlapping!,
                get_activations=True,
                activations_type=int,
                debug=False
            )
        ],
        metrics=EVALUATION_METRICS
    )
    results = exp.run(get_params=False)
    df_results = pd.DataFrame(results)
    df_results.to_excel(f"../results/window{window_size}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx")

Analyzing `RP-CNN` pipeline
Fitting washer dryer model
Epoch 1/10
697/697 [==============================] - 8s 9ms/step - loss: 0.0705 - accuracy: 0.9871
Epoch 2/10
697/697 [==============================] - 6s 8ms/step - loss: 0.0547 - accuracy: 0.9873
Epoch 3/10
697/697 [==============================] - 6s 8ms/step - loss: 0.0528 - accuracy: 0.9873
Epoch 4/10
697/697 [==============================] - 6s 8ms/step - loss: 0.0526 - accuracy: 0.9873
Epoch 5/10
697/697 [==============================] - 6s 9ms/step - loss: 0.0506 - accuracy: 0.9873
Epoch 6/10
697/697 [==============================] - 6s 8ms/step - loss: 0.0509 - accuracy: 0.9873
Epoch 7/10
697/697 [==============================] - 6s 8ms/step - loss: 0.0494 - accuracy: 0.9873
Epoch 8/10
697/697 [==============================] - 6s 8ms/step - loss: 0.0495 - accuracy: 0.9873
Epoch 9/10
697/697 [==============================] - 6s 9ms/step - loss: 0.0483 - accuracy: 0.9873
Epoch 10/10
697/697 [========================

KeyboardInterrupt: 

In [ ]:
df_results.groupby(['model']).agg({
    'f1': ['mean'],
    'training_time': ['mean'],
    'prediction_time': ['mean']
}).sort_values(by=[('f1', 'mean')], ascending=False)

,f1,training_time,prediction_time
,mean,mean,mean
model,,,
RP-VGG16-MLP,0.695798,3.533822,1.645319
RP-CNN,0.670392,6.098470,1.314394


In [ ]:
df_results.groupby(['appliance','model']).agg({
    'f1': ['mean'],
    'training_time': ['mean'],
    'prediction_time': ['mean']
}).sort_values(by=['appliance', ('f1', 'mean')], ascending=[True, False])

f1 training_time prediction_time
                               mean          mean            mean
appliance    model                                               
dish washer  RP-VGG16-MLP  0.496970      3.654790        1.465037
             RP-CNN        0.485271      5.946278        1.093514
fridge       RP-CNN        0.933617      4.351504        1.306546
             RP-VGG16-MLP  0.930617      2.855270        1.456267
microwave    RP-CNN        0.615983      5.480689        1.431913
             RP-VGG16-MLP  0.576890      3.506130        1.650792
washer dryer RP-VGG16-MLP  0.778716      4.119099        2.009181
             RP-CNN        0.646698      8.615411        1.425601

In [ ]:
print('123', flush=True)
print('456', flush=True)
print('789', flush=True)

123
456
789
